In [10]:
##data preparation
import numpy as np
import pandas as pd

np.random.seed(42)

x = np.linspace(0, 1, 100)  
y = np.linspace(0, 1, 100)  
t = np.linspace(0, 5, 100) 

X, Y, T = np.meshgrid(x, y, t)

def temperature_function(x, y, t):
    return 300 + 50 * np.sin(np.pi * x) * np.sin(np.pi * y) * np.exp(-0.1 * t) + 10 * np.random.randn()

temperature_data = temperature_function(X, Y, T)

X_flat = X.flatten()
Y_flat = Y.flatten()
T_flat = T.flatten()
temperature_flat = temperature_data.flatten()

data = pd.DataFrame({
    'x': X_flat,
    'y': Y_flat,
    't': T_flat,
    'temperature': temperature_flat
})

data.to_csv("combustion_data.csv", index=False)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

data = pd.read_csv("combustion_data.csv")

X_tensor = torch.tensor(data[['x', 'y', 't']].values, dtype=torch.float32)
y_tensor = torch.tensor(data['temperature'].values, dtype=torch.float32).view(-1, 1)

class SimplePINN(nn.Module):
    def __init__(self):
        super(SimplePINN, self).__init__()
        self.fc1 = nn.Linear(3, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def compute_physics_loss(model, X, alpha=0.01):
    X.requires_grad = True
    T_pred = model(X)

    
    T_t = torch.autograd.grad(T_pred, X, grad_outputs=torch.ones_like(T_pred), create_graph=True)[0][:, 2]
    T_x = torch.autograd.grad(T_pred, X, grad_outputs=torch.ones_like(T_pred), create_graph=True)[0][:, 0]
    T_y = torch.autograd.grad(T_pred, X, grad_outputs=torch.ones_like(T_pred), create_graph=True)[0][:, 1]
    
    
    T_xx = torch.autograd.grad(T_x, X, grad_outputs=torch.ones_like(T_x), create_graph=True)[0][:, 0]
    T_yy = torch.autograd.grad(T_y, X, grad_outputs=torch.ones_like(T_y), create_graph=True)[0][:, 1]

    
    physics_residual = T_t - alpha * (T_xx + T_yy)
    physics_loss = torch.mean(physics_residual**2)
    return physics_loss

model = SimplePINN()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


ModuleNotFoundError: No module named 'torch'

In [6]:
def train(model, X, y, epochs=1000):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass for data loss
        y_pred = model(X)
        data_loss = data_loss_fn(y_pred, y)

        # Physics-informed loss (heat equation)
        physics_loss = compute_physics_loss(model, X)

        # Combined loss
        loss = data_loss + physics_loss
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            print(f'Epoch {epoch}, Data Loss: {data_loss.item()}, Physics Loss: {physics_loss.item()}')

# Train the model
train(model, X_tensor, y_tensor, epochs=1000)

NameError: name 'model' is not defined

In [7]:
#plot
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    y_pred = model(X_tensor).numpy()

plt.scatter(data['t'], data['temperature'], label='True Data')
plt.scatter(data['t'], y_pred, label='Predicted Data', color='red')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()
plt.show()

NameError: name 'model' is not defined